In [1]:
# Part 2

from collections import defaultdict

def emissionEstimation(filename):
    lines = []
    tags = defaultdict(int) #y to occurence
    count = {}
    labeledCount = defaultdict(dict) #x-y to occurence
    estimates = defaultdict(dict) #x-y to emission
    k = 1
    tag_list = []

    for line in open(filename, 'r'):
        if line:
            lines.append(line.rstrip())
            
    for line in lines:
        vals = line.split(' ') #Split the words and the tags
        if len(vals) == 2:
            x, y = vals[0], vals[1]
        
        if x in count:
            count[x] += 1
        else:
            count[x] = 1
            
        
        #Add the occurance of the tag and labeled words into dictionaries
        if y in labeledCount[x]:
            labeledCount[x][y] += 1
        else:
            labeledCount[x][y] = 1
            
        if y not in tag_list:
            tag_list.append(y)
            
        tags[y] += 1
    
    bottoms = {}
    for y in tags:
        bottoms[y] = float(tags[y] + k)
    
    for y in tags:
        estimates["#UNK#"][y] = k / bottoms[y]
    for x in labeledCount:
        for y in labeledCount[x]:
            top = labeledCount[x][y]
            estimates[x][y] = top/bottoms[y]
            
            
    return count, estimates, tag_list          

In [2]:
count, eEstimates, tags_list = emissionEstimation("train")
print(tags_list)

['O', 'B-positive', 'I-positive', 'B-negative', 'B-neutral', 'I-negative', 'I-neutral']


In [3]:
def predictNaive(filename, estimates):
    sentences = [] # list of sentences
    
    # get best tag for unknown word
    unk_max = None
    unk_max_value = 0
    for y in estimates["#UNK#"]:
        est = estimates["#UNK#"][y]
        if est >= unk_max_value:
            unk_max = y
            unk_max_value = est
            
    new_sentence = [] # list of (x, y)
    sentences.append(new_sentence)
    for line in open(filename, 'r'):
        x = line.rstrip()
        if not x:
            new_sentence = []
            sentences.append(new_sentence)
            continue  
        
        if x not in estimates:
            new_sentence.append((x, unk_max))
        else:
            est_max = None
            est_max_value = 0
            for y in estimates[x]:
                est = estimates[x][y]
                if est >= est_max_value:
                    est_max = y
                    est_max_value = est
                    
            assert(est_max is not None)
            
            new_sentence.append((x, est_max))
    
    return sentences

In [4]:
sentences = predictNaive("dev.in", eEstimates)
print(sentences)
# for i in sentences:
#     print (sentences[0])

[[('Petite', 'O'), ('salle', 'I-neutral'), ('ambiance', 'B-positive'), ('plage', 'I-positive'), ('OlÃ©ronaise', 'I-neutral'), ('.', 'O')], [('La', 'O'), ('salle', 'I-neutral'), ('est', 'O'), ('jolie', 'O'), ('et', 'I-negative'), ('bien', 'O'), ('chauffÃ©e', 'O'), ('.', 'O')], [('Ã‡a', 'O'), ('ne', 'O'), ('les', 'O'), ('vaut', 'O'), ('pas', 'O'), ('.', 'O')], [('A', 'O'), ('ne', 'O'), ('surtout', 'O'), ('pas', 'O'), ('manquer', 'O'), ('.', 'O')], [('Alors', 'O'), ('lÃ', 'I-neutral'), (',', 'O'), ('si', 'O'), ('vous', 'O'), ('voulez', 'O'), ('un', 'O'), ('resto', 'B-neutral'), ('classe', 'O'), (',', 'O'), ('une', 'O'), ('cuisine', 'B-neutral'), ('trÃ¨s', 'O'), ('soignÃ©e', 'I-neutral'), ('et', 'I-negative'), ('rafinÃ©e', 'I-neutral'), ('pour', 'I-neutral'), ('fÃ©ter', 'I-neutral'), ('un', 'O'), ('anniversaire', 'O'), (',', 'O'), ('ou', 'O'), ('une', 'O'), ('grande', 'B-positive'), ('occasion', 'O'), (',', 'O'), ("c'est", 'O'), ('le', 'O'), ('lieu', 'B-positive'), ('IDEAL', 'I-neutral'), 

In [5]:
def compare(count, test_filename, sentences):
    num_of_words = sum(len(s) for s in sentences)
    gold_num_of_words = 0
    num_of_corr_preds = 0
    sent_index = 0
    for line in open(test_filename, 'r'):
        line = line.rstrip()
        if not line:
            sent_index = 1
            continue
        else:
            x, y = line.rstrip().split(' ')
            
        word_index = 0

        if x in count:
            gold_num_of_words += 1
        
        pred_x, pred_y = sentences[sent_index][word_index]
        if pred_y == y:
            num_of_corr_preds += 1
    
    precision = num_of_corr_preds/float(num_of_words)
    recall = num_of_corr_preds/float(gold_num_of_words)
    f = 2/(1/precision + 1/recall)

    return precision, recall, f
        
        
            

In [6]:
precision, recall, f = compare(count, "dev.out", sentences)
print("Precision: %f" % precision)
print("Recall : %f" % recall)
print("F-score: %f" % f)


Precision: 0.905133
Recall : 1.003517
F-score: 0.951789


In [7]:
#Part 3

def transitionEstimation(filename):
    tags = {} #occurances of (y-1)
    nextTags = {} #occurances of transition from (y-1) to y
    estimates = {} #transition estimation
    
    lastState = ""
    state = "#START#"
    
    for line in open(filename, 'r'):
        if state == "#STOP#":
            lastState ="#START#"
        else:
            lastState = state
        
        line = line.rstrip()
        
        if line:
            vals = line.split(' ')
            if len(vals) == 2:
                state = vals[1]
        else:
            state = "#STOP#"
            
        if lastState not in tags:
            tags[lastState] = 1
            nextTags[lastState] = {state: 1}
        else:
            tags[lastState] += 1
            if state in nextTags[lastState]:
                nextTags[lastState][state] += 1
            else:
                nextTags[lastState][state] = 1
                
    for item in tags:
        bottom = float(tags[item])
        estimates[item] = {}
        for nextItem in nextTags[item]:
            top = nextTags[item][nextItem]
            estimates[item][nextItem] = top/bottom

    return(estimates)
            

In [8]:
tEstimates = transitionEstimation("train")
print(tEstimates)

{'I-neutral': {'I-neutral': 0.46511627906976744, 'O': 0.5348837209302325}, 'B-positive': {'I-positive': 0.11851851851851852, '#STOP#': 0.0049382716049382715, 'B-positive': 0.0012345679012345679, 'O': 0.8753086419753087}, 'B-negative': {'#STOP#': 0.002962962962962963, 'O': 0.8296296296296296, 'I-negative': 0.1674074074074074}, 'I-positive': {'I-positive': 0.4696132596685083, '#STOP#': 0.016574585635359115, 'O': 0.5138121546961326}, 'B-neutral': {'I-neutral': 0.20353982300884957, 'O': 0.7964601769911505}, 'O': {'B-neutral': 0.003998041775456919, '#STOP#': 0.06613087467362924, 'B-positive': 0.03027088772845953, 'O': 0.8751631853785901, 'B-negative': 0.02443701044386423}, 'I-negative': {'#STOP#': 0.008583690987124463, 'O': 0.47639484978540775, 'I-negative': 0.5150214592274678}, '#START#': {'B-neutral': 0.009191176470588236, 'O': 0.9031862745098039, 'B-positive': 0.04105392156862745, 'B-negative': 0.04656862745098039}}


In [15]:
from math import log

def viterbi(inpfile, eEstimates, tEstimates, outfile):
    
    count = 0
    count2 = 0
    
    def eachViterbi(sentence):
        nonlocal count
        nonlocal count2
        # returns list of tags
        pi = [] # stores values
        pi_y = [] # stores tags
        
        all_tags = list(tEstimates.keys())
        all_tags.remove("#START#")
        
        # base case
        base_d = {}
        base_d_y = {}
        word = sentence[0]
        pi.append(base_d)
        pi_y.append(base_d_y)
        for tag in all_tags:
            if tag not in tEstimates["#START#"]:
                a = 0.0
            else:
                a = tEstimates["#START#"][tag]
            if word not in eEstimates:
                b = eEstimates["#UNK#"][tag]
            else:
                if tag not in eEstimates[word]:
                    b = eEstimates["#UNK#"][tag]
                else:
                    b = eEstimates[word][tag]
            if a == 0.0:
                base_d[tag] = -100000
            else:
                base_d[tag] = log(a) + log(b)
            base_d_y[tag] = "#START#"
         
        # recursive
        for i in range(1, len(sentence)):
            recur_d = {}
            recur_d_y = {}
            word = sentence[i]
            pi.append(recur_d)
            pi_y.append(recur_d_y)
            for tag_u in all_tags:
                max_value = -1000000
                max_arg = None
                if word not in eEstimates:
                    count += 1
                    b = eEstimates["#UNK#"][tag_u]
                else:
                    if tag_u not in eEstimates[word]:
                        b = eEstimates["#UNK#"][tag]
                    else:
                        b = eEstimates[word][tag_u]
                    
                for tag_v in all_tags:
                    if tag_u not in tEstimates[tag_v]:
                        a = 0.0
                    else:
                        a = tEstimates[tag_v][tag_u]
                    p = pi[i-1][tag_v]
                    if a == 0.0:
                        val = -100000
                    else:
                        val = p + log(a) + log(b)
                    
                    if val >= max_value:
                        max_value = val
                        max_arg = tag_v
                
                assert(max_arg is not None)
                
                recur_d[tag_u] = max_value
                recur_d_y[tag_u] = max_arg
        
        # final case
        n = len(sentence) - 1
        for tag_v in all_tags:
            max_value = -1000000
            max_arg = None

            if "#STOP#" not in tEstimates[tag_v]:
                a = 0.0
            else:
                a = tEstimates[tag_v]["#STOP#"]
            p = pi[n][tag_v]
            if a == 0.0:
                val = -100000
            else:
                val = p + log(a)
            
            if val >= max_value:
                max_value = val
                max_arg = tag_v
            
        assert(max_arg is not None)
        
        before_stop = max_arg
        
        mle_tags = [before_stop]
        current_y = before_stop
        for i in range(len(sentence)-1, 0, -1):
            prev_y = pi_y[i][current_y]

            mle_tags.append(prev_y)
            current_y = prev_y
        
        mle_tags.reverse()
        
        print(pi)
        return mle_tags

    # START OF FUNCTION
    f = open(outfile, 'w')
    observation = [] # each sentence
    
    for line in open(inpfile, 'r'):
        line = line.rstrip()
        if line:
            observation.append(line) # append word to sentence
        else:
            prediction = eachViterbi(observation)
            

            for i in range(len(observation)):
                f.write("%s %s\n" %(observation[i], prediction[i]))
                    
            f.write('\n')
            observation = []
    print(count)
    print(count2)
    f.close()

In [16]:
viterb = viterbi("dev.in", eEstimates, tEstimates, "dev.p3.out")
print(viterb)

[{'I-neutral': -100000, 'B-positive': -9.89605702937395, 'B-negative': -9.585975483178117, 'I-positive': -100000, 'B-neutral': -9.425709782816337, 'O': -8.886524529203971, 'I-negative': -100000}, {'I-neutral': -14.80179301951779, 'B-positive': -17.477843559383672, 'B-negative': -17.507890379135134, 'I-positive': -17.492574890598206, 'B-neutral': -19.872306928099327, 'O': -16.88827677619652, 'I-negative': -16.837132160363918}, {'I-neutral': -21.031092666682973, 'B-positive': -24.31644499657054, 'B-negative': -26.425933358001835, 'I-positive': -23.712252470399324, 'B-neutral': -26.453278637900816, 'O': -20.891330724307814, 'I-negative': -22.964510676173184}, {'I-neutral': -27.260392313848154, 'B-positive': -29.85273135870636, 'B-negative': -30.06681900375829, 'I-positive': -29.688454070253158, 'B-neutral': -31.87711312320317, 'O': -31.195668064294406, 'I-negative': -29.09188919198245}, {'I-neutral': -31.810049789905985, 'B-positive': -41.3964250069082, 'B-negative': -41.426471826659665, 

In [17]:
print(tEstimates)

{'I-neutral': {'I-neutral': 0.46511627906976744, 'O': 0.5348837209302325}, 'B-positive': {'I-positive': 0.11851851851851852, '#STOP#': 0.0049382716049382715, 'B-positive': 0.0012345679012345679, 'O': 0.8753086419753087}, 'B-negative': {'#STOP#': 0.002962962962962963, 'O': 0.8296296296296296, 'I-negative': 0.1674074074074074}, 'I-positive': {'I-positive': 0.4696132596685083, '#STOP#': 0.016574585635359115, 'O': 0.5138121546961326}, 'B-neutral': {'I-neutral': 0.20353982300884957, 'O': 0.7964601769911505}, 'O': {'B-neutral': 0.003998041775456919, '#STOP#': 0.06613087467362924, 'B-positive': 0.03027088772845953, 'O': 0.8751631853785901, 'B-negative': 0.02443701044386423}, 'I-negative': {'#STOP#': 0.008583690987124463, 'O': 0.47639484978540775, 'I-negative': 0.5150214592274678}, '#START#': {'B-neutral': 0.009191176470588236, 'O': 0.9031862745098039, 'B-positive': 0.04105392156862745, 'B-negative': 0.04656862745098039}}


In [28]:
#Part 4

def fbAl(observation, count, eEstimates, tEstimates):
    tags = tags_list
    def getAlpha():
        vals = [{tag:0.0 for tag in tags} for o in observation]
        
        # Base case
        for tag in tags:
            if tag not in tEstimates['#START#']: continue
            vals[0][tag] = tEstimates['#START#'][tag]

        # Recursive case
        for j in range(1, len(observation)):
            for tag1 in tags:
                for tag2 in tags:
                    if tag1 not in tEstimates[tag2]: continue 

                    if observation[j-1] in count:
                        if tag2 not in eEstimates[observation[j-1]]: continue

                        # add if this emission can be found!
                        vals[j][tag1] += vals[j - 1][tag2] * tEstimates[tag2][tag1] * eEstimates[observation[j - 1]][tag2]
                    else:  # if this is #UNK#
                        vals[j][tag1] += vals[j - 1][tag2] * tEstimates[tag2][tag1] * eEstimates['#UNK#'][tag2] 
        return vals
    
    def getBeta():
        vals = [{tag:0.0 for tag in tags} for o in observation]
        
         # Base case
        for tag in tags:
            if '#STOP#' not in tEstimates[tag]: continue 

            if observation[-1] in count:
                if tag not in eEstimates[observation[-1]]: continue 
                vals[-1][tag] = tEstimates[tag]['#STOP#'] * eEstimates[observation[-1]][tag]
            else:  
                vals[-1][tag] = tEstimates[tag]['#STOP#'] * eEstimates['#UNK#'][tag]

        # Recursive case
        for j in reversed(range(len(observation)-1)):
            for tag1 in tags:
                for tag2 in tags:
                    if tag2 not in tEstimates[tag1]: continue 
                    if observation[j] in count:  
                        if tag1 not in eEstimates[observation[j]]: continue 
                        
                        vals[j][tag1] += vals[j + 1][tag2] * tEstimates[tag1][tag2] * eEstimates[observation[j]][tag1]
                    else:  
                        vals[j][tag1] += vals[j + 1][tag2] * tEstimates[tag1][tag2] * eEstimates['#UNK#'][tag1]

        return vals
       
    
    alphas = getAlpha()
    betas = getBeta()    
    
    
    prediction = []
    
    for i in range(len(observation)):
        result = [0.0, '']
        for tag in tags:
            score = alphas[i][tag] * betas[i][tag]
            if score > result[0]:
                result = [score, tag]
        prediction.append(result[1])
    return prediction

In [29]:
def sentiment(inpfile, count, eEstimates, tEstimates, outfile):
    f = open(outfile, 'w')

    observation = []
    for line in open(inpfile, 'r'):
        word = line.rstrip()
        if word:
            observation.append(word)
        else:
            prediction = maxMarginal(observation, count, eEstimates, tEstimates)
            for i in range(len(observation)):
                if prediction:
                    f.write('%s %s\n' % (observation[i], prediction[i]))
                else: 
                    f.write('%s O\n' % observation[i])

            f.write('\n')
            observation = []

    print('Finished writing to file %s' % (outfile))
    return f.close()    

In [30]:
sentiment("dev.in", count, eEstimates, tEstimates, "dev.p4.out")

Finished writing to file dev.p4.out
